In [2]:
import torch
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  #  There are always errors below  shape  atypism

model = torch.load('./mobilenet_v2_best_model.pt')
model.eval()

Unet(
  (encoder): MobileNetV2Encoder(
    (features): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 1), 

In [6]:
torch.jit.script(model)


RuntimeError: 
tuple slice indices must be integer constants:
  File "/home/quiil/anaconda3/envs/pl_wandb/lib/python3.7/site-packages/segmentation_models_pytorch/encoders/_base.py", line 18
    def out_channels(self):
        """Return channels dimensions for each tensor of forward output of encoder"""
        return self._out_channels[: self._depth + 1]
               ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE


In [3]:
scriptedm = torch.jit.trace(model, torch.rand(1, 3, 512, 512).cuda())
# torch.jit.save(scriptedm, "./Unet_scripted.pt")

In [5]:
from torch.utils.mobile_optimizer import optimize_for_mobile

traced_script_module_optimized = optimize_for_mobile(scriptedm)
traced_script_module_optimized._save_for_lite_interpreter("./model.ptl")